<a href="https://colab.research.google.com/github/AncalagonDBlack/source-code-seo-tool/blob/master/Extracting_Backlinks_from_GSC_Ch%C6%B0%C6%A1ng.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://tealfeed.com/methods-extracting-backlinks-google-search-console-cqwwm

https://medium.com/@liviuoltean/seo-how-to-scrape-googles-search-console-for-backlinks-data-3893503cf128

In [ ]:
from bs4 import BeautifulSoup
import requests,time
import re
import pandas as pd
import csv
from google.colab import files
input="yourdomain" #nhập domain zô
headers = {
    "authority": "search.google.com",
    "method":"GET",
    "path":f"/search-console/links?resource_id=https%3A%2F%2F{input}%2F",
    "scheme":"https",  
    "accept":"text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9",
    "accept-encoding":"gzip, deflate, br",
    "accept-language":"en-GB,en-US;q=0.9,en;q=0.8,ro;q=0.7",
    "cache-control":"no-cache",
    "pragma":"no-cache",
    "sec-ch-ua":"navigate",
    "sec-fetch-site":"same-origin",
    "sec-fetch-user":"?1",
    "upgrade-insecure-requests":"1",
    "user-agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/100.0.4896.127 Safari/537.36",
    "x-client-data":"CJG2yQEIpLbJAQjBtskBCKmdygEI94LLAQiUocsBCLi8zAEI6rzMAQjE4cwBCMXhzAEI2OjMAQj/6MwBCPPqzAEInezMAQjm8cwB",
    "sec-ch-ua":'" Not A;Brand";v="99", "Chromium";v="100", "Google Chrome";v="100"',
    "sec-ch-ua-mobile": "?0",
    "sec-fetch-dest": "document",
    "sec-fetch-mode": "navigate"
}
cookies = {
    "__Secure-1PAPISID":"info", #:"[your-info]", cập nhật cái này
    "__Secure-1PSID":"",#:"[your-info]",
    "__Secure-3PAPISID":"",#:"[your-info]",
    "__Secure-3PSID":"",#:"[your-info]",
    "__Secure-3PSIDCC":"",#:"[your-info]", cập nhật cái này
    "1P_JAR":"2022-12-06-02",#:"[your-info]",
    "NID":"", #:"[your-info]",
    "APISID":"",#:"[your-info]",
    #"CONSENT":"[your-info]",
    "HSID":"",#:"[your-info]",
    "SAPISID":"",#:"[your-info]",
    "SID":"",#:"[your-info]",
    "SIDCC":"",#:"[your-info]", cập nhật cái này
    "SSID":"",#:"[your-info]",
    "_ga":"",#:"[your-info]",
    #"OGPC":"[your-info]",
    "OTZ":""#:"[your-info]", ít đổi
}

In [ ]:
genericURL = f"https://search.google.com/search-console/links/drilldown?resource_id=https%3A%2F%2F{input}%2F&type=DOMAIN&target=&domain="
req = requests.get(genericURL, headers=headers, cookies=cookies)
soup = BeautifulSoup(req.content, 'html.parser')
print(soup) #check soup

In [ ]:
regex = re.compile('nJ0sOc wNFy3d ptEsvc*')
g_data = soup.findAll("tr", {"class": regex})
dfdomains = pd.DataFrame()
print(g_data)
countchovui=0

In [ ]:
for externalDomain in g_data:
    #countchovui=countchovui+1
    #if countchovui<311:
    #  continue
    out = re.search(r'<div class="OOHai">(.*?(?=<))', str(externalDomain))
    rawdata=externalDomain.findAll("span", {"class": "PkjLuf"})
    URLList=out.group(1)
    print(URLList)
    urllink=rawdata[1].text
    urldest=rawdata[2].text
    url = f"https://search.google.com/search-console/links/drilldown?resource_id=https%3A%2F%2F{input}%2F&type=DOMAIN&target=&domain={URLList}"
    time.sleep(5)
    request = requests.get(url, headers=headers, cookies=cookies)
    soup_1 = BeautifulSoup(request.content, 'html.parser')
    count_step1=0
    for row in soup_1.findAll("tr", {"class": regex}):
      count_step1=count_step1+1
      if count_step1>100:
        break
      rawdata=row.findAll("span", {"class": "PkjLuf"})
      output = rawdata[0].text
      stripped_output = output.replace("", "")
      urllink_1=rawdata[1].text
      url_secondary = f"https://search.google.com/search-console/links/drilldown?resource_id=https%3A%2F%2F{input}%2F&type=DOMAIN&target={stripped_output}&domain={URLList}"
      time.sleep(3)
      request_secondary = requests.get(url_secondary, headers=headers, cookies=cookies)
      soup_secondary = BeautifulSoup(request_secondary.content, 'html.parser')
      count_step2=0
      for row_1 in soup_secondary.findAll("tr", {"class": regex}):
        count_step2=count_step2+1
        if count_step2>50:
          break
        rawdata2=row_1.findAll("span", {"class": "PkjLuf"})
        output_last = rawdata2[0].text.replace("", "")
        urldest_2=rawdata2[1].text.replace("", "")
        dfdomains = dfdomains.append(pd.DataFrame({"Trang web":[URLList],"Số trang liên kết":[urllink],"Số trang đích":[urldest],"Trang đích":[stripped_output],"Liên kết bên ngoài":[urllink_1],"Trang liên kết":[output_last],"URL đích (nếu khác nhau)":[urldest_2]}))
      #dfdomains = dfdomains.append(pd.DataFrame({"Trang web":"","Số trang liên kết":"","Số trang đích":"","Trang đích":[stripped_output],"Liên kết bên ngoài":[urllink_1],}))
    #dfdomains = dfdomains.append(pd.DataFrame({"Trang web":[URLList],"Số trang liên kết":[urllink],"Số trang đích":[urldest]}))
#domainsList = dfdomains["Trang web"].to_list()
#print(dfdomains) #check list domain xem suất đủ ko 

In [ ]:
dfdomains.to_csv('Backlink.csv', index=False)
files.download('Backlink.csv')